Perceptron convergence theorem &
Gradient descent convergence theorem

when we select appropriate learning rate, we can make error less than set value.

# Loss function

#### 1. MSE: 
$$1/2 \cdot \sum_{k=1}^{n} (yk-tk)^2$$

In [33]:
import numpy as np

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

#### 2. CEE ~ Cross Entropy error

$$-\sum_{i=1}^{n} ti \cdot log(yi)$$

yi is output & ti is answer label.

---

ex)       
binary problem. answer label = {0, 1}

Let y(i) = sigmoid(θ^Txi + θ0).   





y(i) means.. <u>"degree of certainity of 1"</u>

so 1 - y(i) also means.. <u>"degree of certainity of 0"</u>
       
$$
\prod_{i=1}^{n} y_i^{t_i} \cdot (1 - y_i)^{1 - t_i}
$$
we should maximize this.   

by adding minus, we can change it to <u>"loss function minimize problem"</u>   

and also using log, we can simplify total formula.     

<b> it is same form as above! </b>

In [34]:
import numpy as np

def cross_entropy_error(y, t): 
    return -np.sum(t * np.log(y + 1e-7))

In [35]:
t = [0, 0, 1, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.1]

t = np.array(t)
y = np.array(y)

cross_entropy_error(y, t)

0.510825457099338

#### 3. mini-batch


In [52]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from pathlib import Path

# same code as Ch3

path = Path("trainset.csv")

with path.open() as f: 
    lines = f.readlines()

data = [list(map(int, line.strip().split(','))) for line in lines[1:]]

data = np.array(data)
     
x_train = data[:, 1:]

t_train_temp = data[:, 0]  
t_train = np.zeros((x_train.shape[0], 10), dtype=int)

# one-hot coding 
for i, j in enumerate(t_train_temp):
    t_train[i][j] = 1

# check
print("x_train.shape:", x_train.shape)
print("t_train.shape:", t_train.shape)

# print(t_train)

##########################################
path = Path("evalset.csv")

with path.open() as f: 
    lines = f.readlines()

data = [list(map(int, line.strip().split(','))) for line in lines[1:]]

data = np.array(data)

# use as test data
x_test = data[:, 1:]

t_test_temp = data[:, 0] 
t_test = np.zeros((x_test.shape[0], 10), dtype=int)

# check
print("x_test.shape:", x_test.shape)
print("t_test.shape:", t_test.shape)


x_train.shape: (300, 256)
t_train.shape: (300, 10)
x_test.shape: (300, 256)
t_test.shape: (300, 10)


In [37]:
# choice batch's data
train_size = x_train.shape[0]
batch_size = 10

batch_mask = np.random.choice(train_size, batch_size)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [38]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

#### 4. gradient / gradient descent

In gradient descent method, we need to calculate gradient

In [46]:
def gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)

    # 모든 좌표를 (i, j, ...) 형태로 가져오기
    for idx in np.ndindex(x.shape):
        tmp_val = x[idx]

        x[idx] = tmp_val + h
        fxh1 = f(x)

        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val

    return grad

In [40]:
def gradient_descent(f, init_x, lr = 0.01, step_num = 100):

    x = init_x
    
    for i in range(step_num):
        grad = gradient(f, x)
        x -= lr * grad

    return x

In [41]:
# test

def function_1(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_1, init_x = init_x, lr = 0.1, step_num = 100)

array([-6.11110793e-10,  8.14814391e-10])

In [42]:
# GD - Neural Network
import sys, os
sys.path.append(os.pardir)

import numpy as np

class simpleNet:

    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        
        # softmax
        c = np.max(z)
        exp_a = np.exp(z-c)
        sum_exp_a = np.sum(exp_a)
        y = exp_a / sum_exp_a

        loss = cross_entropy_error(y, t)

        return loss


In [43]:
import sys, os
sys.path.append(os.pardir)  

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        # initialize params
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        # sigmoid 
        z1 = 1 / (1 + np.exp(-a1))

        a2 = np.dot(z1, W2) + b2
        # softmax
        c = np.max(a2)
        exp_a = np.exp(a2-c)
        sum_exp_a = np.sum(exp_a)
        y = exp_a / sum_exp_a
        
        return y
        
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = gradient(loss_W, self.params['W1'])
        grads['b1'] = gradient(loss_W, self.params['b1'])
        grads['W2'] = gradient(loss_W, self.params['W2'])
        grads['b2'] = gradient(loss_W, self.params['b2'])
        
        return grads

In [44]:
# size test
net = TwoLayerNet(input_size=256, hidden_size = 50, output_size=10)

print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(256, 50)
(50,)
(50, 10)
(10,)


In [56]:
# SGD
import numpy as np

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

# hyperparameters
iteration_num = 100
train_size = x_train.shape[0] # 300
batch_size = 300
learning_rate = 0.1

network = TwoLayerNet(input_size = 256, hidden_size = 50, output_size = 10)

for i in range(iteration_num):

    batch_mask = np.random.choice(train_size, batch_size)

    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # calculate gradient
    grad = network.numerical_gradient(x_batch, t_batch)

    # update params
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)

        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.29333333333333333, 0.0
train acc, test acc | 0.27, 0.0033333333333333335
train acc, test acc | 0.56, 0.12333333333333334
train acc, test acc | 0.6866666666666666, 0.13666666666666666
train acc, test acc | 0.6233333333333333, 0.2833333333333333


KeyboardInterrupt: 